<a href="https://colab.research.google.com/github/Malik-Zubair123/Gemma2_Hackathon/blob/main/Edit_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pip as pip


pip install transformers torch

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for torch


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

ModuleNotFoundError: No module named 'torch'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import traceback


class TranscriptSummarizer:
    def __init__(self, model_name="google/gemma-2b", token="Your_Token_Here"):
        """
        Initialize the summarization model and tokenizer with Hugging Face authentication.

        Args:
            model_name (str): Hugging Face model to use for summarization.
            token (str): Hugging Face API token.
        """
        try:
            # Set the Hugging Face token for authentication
            from huggingface_hub import login
            login(token)  # Authenticate using the token

            # Load tokenizer and model with specific configurations
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
            self.model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)

            # Set pad token if not already set
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.model.config.eos_token_id

        except Exception as e:
            print(f"Model loading error: {e}")
            traceback.print_exc()
            raise

    def preprocess_text(self, text, max_length=2048):
        """
        Preprocess input text for summarization with improved truncation.

        Args:
            text (str): Input text to preprocess.
            max_length (int): Maximum token length to allow.

        Returns:
            str: Cleaned and processed text.
        """
        # Remove extra whitespaces
        cleaned_text = ' '.join(text.split())

        # Truncate text to manageable length
        tokens = self.tokenizer.encode(
            cleaned_text,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        return self.tokenizer.decode(tokens[0], skip_special_tokens=True)

    def generate_summary(self, text, max_new_tokens=150):
        """
        Generate summary using the loaded model with enhanced error handling.

        Args:
            text (str): Text to summarize.
            max_new_tokens (int): Maximum number of new tokens to generate.

        Returns:
            str: Generated summary.
        """
        try:
            # Preprocess the text with length control
            processed_text = self.preprocess_text(text)

            print(f"Processed Text Size: {len(processed_text)} characters")

            # Construct a prompt that explicitly requests a summary
            summary_prompt = f"Summarize the following text concisely:\n{processed_text}\n\nSummary:"

            # Prepare inputs with error checking
            inputs = self.tokenizer(
                summary_prompt,
                return_tensors="pt",
                truncation=True,
                max_length=2048,
                padding=True
            )

            # Generate summary with more robust parameters
            summary_ids = self.model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7,
                pad_token_id=self.tokenizer.pad_token_id
            )

            # Decode the summary
            summary = self.tokenizer.decode(
                summary_ids[0][inputs['input_ids'].shape[1]:],  # Only decode new generated tokens
                skip_special_tokens=True
            )

            return summary.strip()

        except Exception as e:
            print(f"Detailed Summary Generation Error: {e}")
            traceback.print_exc()
            return f"Could not generate summary. Error: {str(e)}"


# Function to handle different input scenarios
def summarize_text(input_text=None, file_path=None, max_new_tokens=150):
    """
    Flexible function to summarize text from different sources.

    Args:
        input_text (str, optional): Direct text input.
        file_path (str, optional): Path to text file.
        max_new_tokens (int, optional): Maximum new tokens in summary.

    Returns:
        str: Generated summary.
    """
    # Initialize summarizer
    summarizer = TranscriptSummarizer()

    # Determine text source
    if input_text:
        text = input_text
    elif file_path:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
        except Exception as e:
            print(f"Error reading file: {e}")
            return f"File reading error: {e}"
    else:
        return "No text provided. Please give input text or file path."

    # Generate and return summary
    return summarizer.generate_summary(text, max_new_tokens)


# Example Usage
def main():
    # Option 1: Directly input text
    sample_text = """
    Key Advantages of Plain Text Files:
    Plain text will never require a subscription, lock away features, or go out of business. It's free and here forever.

    Flexibility: Open Them Anywhere: Plain text files are the most flexible file format we have. They can be opened by hundreds, if not thousands, of applications. Since they are a basic component of computers, you can even open them on the computer command line.
    No New Tools: We all get excited by new stuff, and for productivity junkies, we perk up with excitement to try out a new piece of software. Unfortunately tools come and go, and switching system can be wasteful and counterproductive. Switching to plain text means you never need to migrate to a new tool.
    Portability: By portability I mean that your files can be moved to and from different operating systems, platforms and devices and you can still open them. Whether you are on a Mac, Linux, Windows or some future tool, you'll be able to open and edit your plain text files there.
    Future-Proof: A while back I discovered several writings I did in high school using Windows 95 and a version of Microsoft Word. While I was able to eventually open the files, the formatting had been lost. This revealed a potential danger in locking your words into a format that may not be around forever. Fortunately, while other file formats may come and go, plain text files will remain.
    """

    # Generate summary from text
    summary = summarize_text(input_text=sample_text, max_new_tokens=100)
    print("Generated Summary:")
    print(summary)


if __name__ == "__main__":
    main()


UI Libraries:

In [ ]:
!pip install streamlit
!pip install pyngrok


UI

In [ ]:
%%writefile app.py
import importlib.util
import sys
import os

# Define the file path dynamically
file_path = '/content/model.py'

# Check if the file exists before loading
if os.path.exists(file_path):
    spec = importlib.util.spec_from_file_location("model", file_path)
    model = importlib.util.module_from_spec(spec)
    sys.modules["model"] = model
    spec.loader.exec_module(model)
else:
    raise FileNotFoundError(f"{file_path} not found")

from model import TranscriptSummarizer
import streamlit as st
from io import StringIO
import PyPDF2

# Set up the Streamlit app configuration
st.set_page_config(page_title="Meeting Summarizer AI", page_icon=":📝", layout="wide", initial_sidebar_state="auto", menu_items=None)

# Header and Subheader
st.header("Meeting Summarizer AI", anchor="header")
st.subheader("Upload transcripts and get actionable insights in seconds!", anchor="subheader")

# File upload widget
uploadFile = st.file_uploader("Upload your meeting transcript", type=["txt", "pdf"], accept_multiple_files=True)

# Text input area
transcript_input = st.text_area("Enter your transcript here:", placeholder="Meeting minutes...", height=200)

# Initialize variable for the transcript content
input_text = None

summarizer = TranscriptSummarizer()

# Function to handle PDF text extraction
def extract_text_from_pdf(pdf_file):
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        return text
    except Exception as e:
        st.error(f"Error extracting text from PDF: {e}")
        return None

# Process file upload or use text input
if uploadFile is not None:
    input_text = ""
    for file in uploadFile:
        if file.type == "text/plain":
            input_text += file.read().decode("utf-8")
        elif file.type == "application/pdf":
            # Extract text from PDF
            pdf_text = extract_text_from_pdf(file)
            if pdf_text:
                input_text += pdf_text
else:
    input_text = transcript_input

# Button to trigger summarization
summarizeButton = st.button("Summarize")

# If the button is clicked, generate and display summary
if summarizeButton:
    if input_text:
        try:
            # Step 1: Preprocess the text
            preprocessed_text = summarizer.preprocess_text(input_text)

            # Display preprocessed text (for debugging or validation)
            st.subheader("Preprocessed Text:")
            st.write(preprocessed_text)

            # Step 2: Generate the summary
            summary = summarizer.generate_summary(input_text)

            # Display the generated summary
            st.subheader("Generated Summary:")
            st.write(summary)

        except Exception as e:
            st.error(f"Error during summarization: {str(e)}")
    else:
        st.warning("Please provide a transcript either by uploading a file or entering text.")

# Download button for the generated summary
if input_text and summarizeButton:
    st.download_button(
        label="Download Summary",
        data=summary,
        file_name="summary.txt",
        mime="text/plain"
    )

To run it.

In [ ]:
from pyngrok import ngrok
import os

# Set ngrok authentication token (replace with your token)
ngrok.set_auth_token("Your_ngrok_token")

# Open the tunnel to Streamlit port 8501
public_url = ngrok.connect(8501)

# Start Streamlit app in the background
os.system("streamlit run model.py &")

# Print the public URL for accessing the app
print(f"Streamlit app is running at: {public_url}")
